# Business Understanding
Briefly restate the project’s purpose and goals.

Define the research/prediction question (e.g., “Can we predict a job’s salary based on location, company, and job title?”).

Describe why the problem is important or useful.

### Data Preview

In [ ]:
# Panda dataframe here to showcase original data from csv

# Data Cleaning

### Variable Characterisation
- Have all fields defined in this block

### Handling Missing Data

In [ ]:
# Code here to check for any missing values to address

Then explain any missing values in this cell and how to approach

In [ ]:
# Then code here to handle missing values, if we have any. Follow TitanicDataCleaning.ipynb notes

# Data Exploration
This step includes Univariate, Bivariate, and Multivariate Analysis. Like CA1 but in code this time

### Univariate Analysis

In [ ]:
# Univariate analysis

### Bivariate Analysis

In [ ]:
# Bivariate analysis

### Multivariate Analysis

In [ ]:
# Multivariate analysis

# Feature Engineering

### Response variable
- Explain that we're doing salary

### Variable Selection and Creation

In [ ]:
# Then code for creating or dropping any new variables we may need

Then explain why we created/removed some

### Importance and Multicolinearity

In [ ]:
# Code here of different correlation graphs like in demoTitanicv2025v1.ipynb

Then explain output

# Predictive Modelling
State what we will use. Best to follow notes

# Findings
Summarize main results and conclusions.

Highlight key visualizations that support findings.

Connect insights back to your original business question.

Discuss limitations and possible improvements.

Probably best to split above 4 into 4 separate blocks

# Team Contributions

### Theo's Contributions
- Provided basic code to extract data from Jobs.ie and export it to a csv file
- Created initial template for both DataMining and DataAnalyses files 

### Oisín's Contributions
- Created README.md file to explain project
- Researched potential websites to use for project